In [ ]:
# For colab only
# %pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
# %pip install transformers accelerate datasets
# %pip install tqdm
# %pip install numpy
# %pip install scikit-learn
# %pip install opencv-python
# %pip install tensorboardX
# %pip install pandas

In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import SubsetRandomSampler, DataLoader
from torchvision.transforms import GaussianBlur
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import gaussian_filter, zoom, mean as ndimage_mean
from skimage.transform import resize
import cv2


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Functions for degradation

In [ ]:
# Functions for pixellation

def upsample_nearest(image, new_width, new_height):

    old_height, old_width = image.shape[-2:]
    scale_x = new_width / old_width
    scale_y = new_height / old_height

    # Create the coordinate grid for the new image
    x_indices = (np.arange(new_width) / scale_x).astype(int)
    y_indices = (np.arange(new_height) / scale_y).astype(int)

    # Use advanced indexing to map the input pixels to the new grid
    upsampled_image = image[:,y_indices[:, None], x_indices]
    return upsampled_image

def downsample_to_fixed_size(image, target_size):

    num_channels, input_height, input_width = image.shape
    target_rows, target_cols = target_size

    # Calculate the scaling factors
    row_scale = input_height / target_rows
    col_scale = input_width / target_cols

    # Create an empty array for the downsampled image
    downsampled_image = np.zeros((num_channels,target_rows, target_cols), dtype=np.float32)

    for row in range(target_rows):
        for col in range(target_cols):
            # Determine the boundaries of the region in the original image
            row_start = int(row * row_scale)
            row_end = int((row + 1) * row_scale)
            col_start = int(col * col_scale)
            col_end = int((col + 1) * col_scale)

            # Extract the region and compute the average for each channel
            region = image[:,row_start:row_end, col_start:col_end]
            downsampled_image[:,row, col] = region.mean(axis=(1, 2))

    return downsampled_image.astype(image.dtype)

# Taken from https://github.com/hendrycks/robustness/blob/master/ImageNet-C/imagenet_c/imagenet_c/corruptions.py
def snow(x0, severity=1):
    c = [(0.1, 0.3, 3, 0.5, 10, 4, 0.8),
         (0.2, 0.3, 2, 0.5, 12, 4, 0.7),
         (0.55, 0.3, 4, 0.9, 12, 8, 0.7),
         (0.55, 0.3, 4.5, 0.85, 12, 8, 0.65),
         (0.55, 0.3, 2.5, 0.85, 12, 12, 0.55)][severity - 1]

    x0 = np.array(x0, dtype=np.float32) / 255.
    snow_layer = np.random.normal(size=x0.shape[:2], loc=c[0], scale=c[1])  # [:2] for monochrome

    #snow_layer = clipped_zoom(snow_layer[..., np.newaxis], c[2])
    snow_layer[snow_layer < c[3]] = 0

    snow_layer = np.clip((snow_layer.squeeze(), 0, 1) * 255).astype(np.uint8)

    #snow_layer.motion_blur(radius=c[4], sigma=c[5], angle=np.random.uniform(-135, -45))

    snow_layer = cv2.imdecode(np.fromstring(snow_layer.make_blob(), np.uint8),
                              cv2.IMREAD_UNCHANGED) / 255.
    snow_layer = snow_layer[..., np.newaxis]

    x0 = c[6] * x0 + (1 - c[6]) * np.maximum(x0, cv2.cvtColor(x0, cv2.COLOR_RGB2GRAY).reshape(224, 224, 1) * 1.5 + 0.5)
    return np.clip(x0 + snow_layer + np.rot90(snow_layer, k=2), 0, 1) * 255





In [ ]:
class coldDiff:
    def __init__(self, steps=300, size=28, loss_type='L1',degradation_type='blur'):
    
        self.steps = steps
        self.size = size
        self.loss_type = loss_type

        self.beta_schedule = self.get_beta_schedule(self.steps)
        self.alpha = 1.0 - self.beta_schedule
        alphas = torch.cat([torch.tensor([1.]), self.alpha])  # Add a leading 1.0 to the alphas tensor
        self.alpha_hat = torch.cumprod(alphas, dim=0).to(device)

        # Degradation type can be 'blur', 'pixellate', 'inpainting' or 'snow'
        if degradation_type == 'blur':
            self.degradation = self.blur
        elif degradation_type == 'pixellate':
            self.degradation = self.pixellate
        elif degradation_type == 'inpainting':
            self.degradation = self.inpainting
        elif degradation_type == 'snow':
            self.degradation = self.snow
        elif degradation_type == 'gaussian':
            self.degradation = self.gaussian_noise
        else:
            raise ValueError("Invalid degradation type. Choose from 'blur', 'pixellate', 'inpainting' or 'snow'.")

    # SAMPLING FUNCTIONS
    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.steps, size=(n,)) #Should this be self.steps+1?
    
    def sample(self, model, batch_size,initial_image='real_degraded', data_loader=None, gmm=None):
        output_shape = (batch_size, 1, 28, 28)
        t = self.steps
        with torch.no_grad():
            model.eval()
            #x_prev = x_t
            # black image
            if initial_image == 'black':
                x_prev = torch.zeros(output_shape).to(device)

            if initial_image == 'random':
                # random and degraded
                dummy_x0 = torch.randn(output_shape).to(device)  # Or: real_batch_from_train_loader
                timestep_tensor = torch.tensor([self.steps] * batch_size).to(device)
                x_prev = self.degradation(dummy_x0, timestep_tensor)\
            
            if initial_image == 'real_degraded':
                if data_loader is None:
                    raise ValueError("data_loader is not provided. Please provide a DataLoader for sampling.")
                real_batch, _ = next(iter(data_loader))  # get real MNIST digits
                real_batch = real_batch.to(device)
                real_batch = real_batch[:batch_size]  # in case batch size mismatch
                timestep_tensor = torch.tensor([self.steps] * real_batch.shape[0]).to(device)
                x_prev = self.degradation(real_batch, timestep_tensor)

            if initial_image == 'gmm':
                # Gaussian Mixture Model
                if gmm is None:
                    raise ValueError("GMM is not provided. Please provide a GMM model for sampling.")
                # Sample from the GMM
                gmm_samples, _ = gmm.sample(batch_size)  # Sample from the GMM
                gmm_samples = gmm_samples.astype(np.float32)
                gmm_samples = torch.from_numpy(gmm_samples)  # Convert to tensor
                gmm_samples = gmm_samples.view(batch_size, 1, self.size, self.size)  # Reshape to the desired output shape
                gmm_samples = gmm_samples.to(device)  # Move to the appropriate device
                # Set the initial image to the GMM samples
                x_prev = gmm_samples
                
                


            for s in range(t, 0, -1):
                s_ = (torch.ones(batch_size)*s).long().to(device)
                pred_x0 = model(x_prev, s_)
                x_prev = x_prev - self.degradation(pred_x0, s_) + self.degradation(pred_x0, s_-1)
        return x_prev

    def sample_ddpm(self, model, batch_size):
        model.eval()
        # Algo 2 - Sampling
        with torch.no_grad():
            x = torch.randn((batch_size, 1, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(batch_size) * i).long().to(self.device)
                predicted_noise = model(x, t)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i>1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1/torch.sqrt(alpha)*(x-((1-alpha)/torch.sqrt(1-alpha_hat))*predicted_noise)+torch.sqrt(beta)*noise
            model.train()
            x = (x.clamp(-1,1)+1)/2
            x = (x*255).type(torch.uint8)
            return x

    # Normal gaussian noise degradation
    def gaussian_noise(self, x0, t):
        """if len(t.shape) == 1:
            t = t[:, None, None, None]

        sqrt_alpha_bar = self.alpha_bar[t.squeeze().long().clamp(max=self.steps)].sqrt().to(x0.device)[:, None, None, None]
        sqrt_one_minus_alpha_bar = (1 - self.alpha_bar[t.squeeze().long().clamp(max=self.steps)]).sqrt().to(x0.device)[:, None, None, None]

        noise = torch.randn_like(x0)
        
        return sqrt_alpha_bar * x0 + sqrt_one_minus_alpha_bar * noise
        """
        

        e = torch.randn_like(x0)
        x_t = x0*torch.sqrt(self.alpha_hat[t])[:, None, None, None] + torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]*e 
        return x_t

   

    # DEGRADATION FUNCTIONS
    def blur(self, x0, t, base_sigma=0.33):
        sigma_t = torch.sqrt(t*base_sigma**2).detach().cpu().numpy()
        x0_cpu = x0.detach().cpu().numpy()
        blurred_imgs = [gaussian_filter(x0_cpu[i], sigma=(0, sigma_t[i], sigma_t[i])) for i in range(len(sigma_t))]
        return torch.from_numpy(np.stack(blurred_imgs)).to(device)
        # return GaussianBlur(5, sigma_t)

    def pixellate(self,x0,t,final_resolution=4): # Super resolution in the paper

        original_w,original_h = x0.shape[2],x0.shape[3]
                
        w,h = original_w,original_h
        pixellated_imgs = x0.detach().cpu().numpy()

        # Convert to list of numpy arrays
        pixellated_imgs = [pixellated_imgs[i].astype(np.float32) for i in range(len(pixellated_imgs))]
        while w > final_resolution and h > final_resolution:
            if(w//2 < final_resolution and h//2 < final_resolution):
                target_width = final_resolution
                target_height = final_resolution
            else:
                target_width = w//2
                target_height = h//2
            # Downsample the image using average pooling
            for i in range(len(pixellated_imgs)):
                if t[i] > 0:
                    
                    pixellated_imgs[i] = downsample_to_fixed_size(pixellated_imgs[i], (target_height, target_width))
                    #print(f"pixellated_imgs[{i}].shape:{pixellated_imgs[i].shape}, target_width:{target_width}, target_height:{target_height}")
             
            w,h = w//2,h//2
            
            t = t - 1    
        # Up sample the image to the original resolution (original_w,original_h) using nearest neighbour interpolation

        pixellated_imgs = [upsample_nearest(pixellated_imgs[i], original_w, original_h) for i in range(len(pixellated_imgs))]


        return torch.from_numpy(np.stack(pixellated_imgs)).to(device)
    
    def inpainting(self,x0,t,base_variance=1): #Base variance is beta in the paper
        w,h = x0.shape[2],x0.shape[3]
        center_x,center_y = np.random.randint(0,w, size = x0.shape[0]),np.random.randint(0,h,x0.shape[0])
        variance = base_variance + 0.5*t.detach().cpu().numpy()
        # 2d gaussian curve with center at rand_x,rand_y and peak value = 1, discretized 

        x0 = x0.detach().cpu().numpy()
        gaussian_mask = np.zeros_like(x0, dtype=float)

        y, x = np.meshgrid(np.arange(h), np.arange(w), indexing='ij')
        gaussian = np.stack([np.exp(-((x - center_x[i])**2 + (y - center_y[i])**2) / (2 * variance[i])) for i in range(len(x0))])
        gaussian = 1 - gaussian

        # Normalize the gaussian mask
        gaussian = gaussian / np.max(gaussian, axis=(1,2), keepdims=True)

        # Add extra channel dimension as 2nd dimension
        gaussian = gaussian[:,None,:,:]
    
        # Apply the mask to the image
        inpainted_imgs = x0 * gaussian
     
        return torch.from_numpy(inpainted_imgs).float().to(device)

    def snow(self,x0, severity=1):
        c = [(0.1, 0.3, 3, 0.5, 10, 4, 0.8),
            (0.2, 0.3, 2, 0.5, 12, 4, 0.7),
            (0.55, 0.3, 4, 0.9, 12, 8, 0.7),
            (0.55, 0.3, 4.5, 0.85, 12, 8, 0.65),
            (0.55, 0.3, 2.5, 0.85, 12, 12, 0.55)][severity - 1]

        x0 = x0.detach().cpu().numpy() / 255.
    
        snow_layer = np.random.normal(size=x0.shape[:2], loc=c[0], scale=c[1])  # [:2] for monochrome

        #snow_layer = clipped_zoom(snow_layer[..., np.newaxis], c[2])
        snow_layer[snow_layer < c[3]] = 0

        snow_layer = (np.clip(snow_layer.squeeze(), 0, 1) * 255).astype(np.uint8)

        snow_layer = cv2.imdecode(np.fromstring(snow_layer.make_blob(), np.uint8),
                                cv2.IMREAD_UNCHANGED) / 255.
        snow_layer = snow_layer[..., np.newaxis]

        x0 = c[6] * x0 + (1 - c[6]) * np.maximum(x0, cv2.cvtColor(x0, cv2.COLOR_RGB2GRAY).reshape(224, 224, 1) * 1.5 + 0.5)
        snowified_images =  np.clip(x0 + snow_layer + np.rot90(snow_layer, k=2), 0, 1) * 255
        
        return torch.from_numpy(snowified_images).to(device)

    ## Scheduling 
    def get_beta_schedule(self, timesteps, start=1e-4, end=0.02, schedule_type='linear'):
        if schedule_type == 'linear':
            return torch.linspace(start, end, timesteps)
        elif schedule_type == 'cosine':
            steps = torch.arange(timesteps + 1, dtype=torch.float32)
            f = lambda t: torch.cos(((t / timesteps) + 0.008) / 1.008 * torch.pi / 2) ** 2
            alphas_bar = f(steps) / f(torch.tensor(0.0))
            betas = 1 - (alphas_bar[1:] / alphas_bar[:-1])
            return torch.clip(betas, 1e-5, 0.999)
        else:
            raise ValueError("Unknown schedule type")

    

    


# Network definition

In [ ]:
class UNet(nn.Module):
    def __init__(self, c_in=1, c_out=1, time_dim=256, device=device):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, 14)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, 7)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, 4)
        
        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 512)
        self.bot3 = DoubleConv(512, 256)
        
        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, 7)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, 14)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64,28)
        self.outc = nn.Conv2d(64, c_out, kernel_size=1)
        
    # Sinosoidal encoding - further read
    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (10000**(torch.arange(0, channels, 2, device=self.device).float() / channels))
        
        pos_enc_a = torch.sin(t.repeat(1, channels // 2)*inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels//2)*inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc
    
    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)
        
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)
        
        # Bottleneck
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        x4 = self.bot3(x4)
        
        # Decoder
        #print(x4.size())
        #print(x3.size())
        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        return self.outc(x)  

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.GroupNorm(1, mid_channels)
        self.act = nn.GELU() ## Try Relu, leakyReLU
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.norm2 = nn.GroupNorm(1, out_channels)
        self.residual = residual
        
    def forward(self, x):
        x2 = self.conv1(x)
        x2 = self.norm1(x2)
        x2 = self.act(x2)
        x2 = self.conv2(x2)
        x2 = self.norm2(x2)
        if self.residual:
            return self.act(x+x2)
        else:
            return x2

In [ ]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.maxPool = nn.MaxPool2d(2)
        self.doubleConv1 = DoubleConv(in_channels, in_channels, residual=True)
        self.doubleConv2 = DoubleConv(in_channels, out_channels)
        
        self.act = nn.SiLU()
        self.linear = nn.Linear(emb_dim, out_channels)
        
    def forward(self, x, t):
        x = self.maxPool(x)
        x = self.doubleConv1(x)
        x = self.doubleConv2(x)
        #print(x.size())
        emb = self.act(t)
        emb = self.linear(emb)[:, :, None, None].repeat(1,1,x.shape[-2], x.shape[-1])
        #print(emb.size())
        
        return x+emb  

In [ ]:
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        
        #self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
        self.doubleConv1 = DoubleConv(in_channels, in_channels, residual=True)
        self.doubleConv2 = DoubleConv(in_channels, out_channels, in_channels//2)
        self.act = nn.SiLU()
        self.linear = nn.Linear(emb_dim, out_channels)
        
    def forward(self, x, skip_x, t):
        #print(x.size())
        x = self.up(x)
        #print(x.size())
        if x.shape[-2:] != skip_x.shape[-2:]:
            x = nn.functional.interpolate(x, size=skip_x.shape[-2:], mode='bilinear', align_corners=True)
            #print(x.size())
        x = torch.cat([skip_x, x], dim=1)
        x = self.doubleConv1(x)
        x = self.doubleConv2(x)
        emb = self.act(t)
        emb = self.linear(emb)[:, :, None, None].repeat(1,1,x.shape[-2], x.shape[-1])
        return x+emb

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.linear = nn.Linear(channels, channels)
        self.act = nn.GELU()
    
    def forward(self, x):
        b, c, h, w = x.shape
        x = x.view(b, c, h*w).permute(0,2,1)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        x = self.ln(attention_value)
        x = self.linear(x)
        x = self.act(x)
        x = self.linear(x)
        attention_value = x + attention_value
        
        return attention_value.permute(0, 2, 1).view(b, c, h, w)

## Function to plot images

In [ ]:
def plot_images(images):
    plt.figure(figsize=(32, 32))
    plt.imshow(torch.cat([torch.cat([i for i in images.detach().cpu()], dim=-1)], dim=-2).permute(1,2,0).cpu(), cmap='gray')
    plt.show()

In [ ]:
import torchvision

transforms = torchvision.transforms.Compose([
    #torchvision.transforms.Resize(80),
    #torchvision.transforms.RandomResizedCrop(args.img_size, scale=(0.8, 1.0)),
    torchvision.transforms.Resize((28, 28)),    # Resize to 28x28 for MNIST
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5,), (0.5,)) 
])

In [ ]:
np.random.seed(32)

def split_indices(n, val_pct):
    n_val = int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

## Training setup

In [ ]:
EPOCHS = 100
batch_size = 32
image_size = 28
learning_rate = 1e-3
loss_type = 'L1' # 'L1', 'L2', 'SmoothL1', 'Huber'
sampling_type = 'gmm' # 'random', 'black', 'real_degraded', 'gmm'

# DEGRADATION TYPE
degradation_type = 'blur' # 'blur', 'pixellate', 'inpainting', 'snow', 'gaussian'

# Steps for diffusion process - changes based on the function used
# For pixellate, steps = 4 (MNIST) or 6 (CIFAR10)
steps = 200

FASHION_MNIST = False

# Import dataset
if (FASHION_MNIST):
    dataset = torchvision.datasets.FashionMNIST(root="./data", train=True, transform=transforms, download=True)
    test_dataset = torchvision.datasets.FashionMNIST(root="./data", train=False, transform=transforms, download=True)
else:
    dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms, download=True)
    test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms, download=True)

# LIMIT THE DATASET
LIMIT = True
if LIMIT:
    num_samples = 6000
    num_test_samples = num_samples // 6

    # Randomly select indices for train and test
    all_indices = np.random.permutation(len(dataset))
    train_indices = all_indices[:num_samples]
    test_indices = np.random.permutation(len(test_dataset))[:num_test_samples]

    dataset = torch.utils.data.Subset(dataset, train_indices)
    test_dataset = torch.utils.data.Subset(test_dataset, test_indices)


    print("LIMITED DATASET")
    print(len(dataset))
    print(len(test_dataset))

else:
    print("FULL DATASET")
    print(len(dataset))
    print(len(test_dataset))
# dataset = torch.utils.data.Subset(dataset, range(1000))
# test_dataset = torch.utils.data.Subset(test_dataset, range(1000))


train_indices, val_indices = split_indices(len(dataset), 0.2)
train_sampler = SubsetRandomSampler(train_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
val_sampler = SubsetRandomSampler(val_indices)
val_loader = DataLoader(dataset, batch_size, sampler=val_sampler)

model = UNet().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)

if loss_type == 'L1':
    loss_fn = nn.L1Loss()
elif loss_type == 'L2':
    loss_fn = nn.MSELoss()
elif loss_type == 'SmoothL1':
    loss_fn = nn.SmoothL1Loss()
elif loss_type == 'Huber':
    loss_fn = nn.HuberLoss()
elif loss_type == 'BCE':
    loss_fn = nn.BCEWithLogitsLoss()
else:
    raise ValueError("Invalid loss type. Choose from 'L1', 'L2', 'SmoothL1', 'Huber' or 'BCE'.")
    
diffusion = coldDiff(size=image_size, steps = steps, degradation_type=degradation_type)
noise_function = diffusion.degradation
length = len(train_loader)
print(length)

train_losses = []
val_losses = []



In [ ]:
# Plot histograms
def plot_histograms(images, figsize=(32,32)):
    """
    Plot histograms of pixel values for a batch of images in a single row
    Args:
        images: Tensor of shape (batch_size, channels, height, width)
        figsize: Size of the figure to plot
    """
    # Move images to CPU and convert to numpy array
    if torch.is_tensor(images):
        images = images.cpu().detach().numpy()
    
    batch_size = images.shape[0]
    
    # Create figure with subplots in a single row
    # Make each subplot square by setting height equal to width
    subplot_width = figsize[0] / batch_size
    fig, axes = plt.subplots(1, batch_size, figsize=(figsize[0], subplot_width))
    
    # Plot histogram for each image
    for i in range(batch_size):
        img = images[i].reshape(-1)  # Flatten single image to 1D
        axes[i].hist(img, bins=50, density=True)
        axes[i].set_ylim(0, 1)  # Limit height
        axes[i].axis('off')  # Remove axes
    
    plt.tight_layout()
    plt.show()


### Visualize the dataset and the degradation methods

In [ ]:
for x, _ in train_loader:
    x = x.to(device)
    t = diffusion.sample_timesteps(x.shape[0])
    print(t,t.shape)
    # Plot the images
    plot_images(x)
    x_t = diffusion.inpainting(x, t)
    plot_images(x_t)
    x_t = diffusion.pixellate(x, t)
    plot_images(x_t)
    x_t = diffusion.blur(x, t)
    plot_images(x_t)
    #x_t = diffusion.snow(x)
    #plot_images(x_t)
    x_t = diffusion.gaussian_noise(x, t)
    plot_images(x_t)
    
    break

In [ ]:
# Visualize one image degraded to n time steps

factor = steps// batch_size

time_steps = range(0, steps, factor)
time_steps = time_steps[:batch_size]
t = torch.tensor(time_steps).to(device)
print(t,t.shape)
for x, _ in train_loader:
    # Repeat the same image
    x = x[0].unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)
    x_t = diffusion.degradation(x, t,base_sigma=0.33)
    plot_images(x_t)
    break

# Visualize the degradation level per time step linearly

In [ ]:
def psnr(original, degraded):
    # Convert tensors to numpy arrays if needed
    if torch.is_tensor(original):
        original = original.detach().cpu().numpy()
    if torch.is_tensor(degraded):
        degraded = degraded.detach().cpu().numpy()
        
    # Ensure same shape
    #print(original.shape, degraded.shape)
    assert original.shape == degraded.shape, "Images must have same dimensions"
    
    mse = np.mean((original - degraded) ** 2)
    if mse == 0:
        return float('inf')
    
    # Assume images are normalized to [0,1] range
    PIXEL_MAX = 1.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

In [ ]:
factor = steps// batch_size
time_steps = range(0, steps, factor)
time_steps = time_steps[:batch_size]

t = torch.tensor(time_steps).to(device)
print(t)
for x, _ in train_loader:
    x = x[0]
    x = x.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)

    x_t = diffusion.blur(x, t,base_sigma=0.33)
    variance_blur = torch.var(x_t, dim=(1,2,3))
    psnr_blur = []
    for i in range(batch_size):
        psnr_blur.append(psnr(x[0], x_t[i]))
    plot_images(x_t)
    plot_histograms(x_t)

    x_t = diffusion.gaussian_noise(x, t)
    variance_noise = torch.var(x_t, dim=(1,2,3))
    psnr_noise = []
    for i in range(batch_size):
        psnr_noise.append(psnr(x[0], x_t[i]))
    plot_images(x_t)
    plot_histograms(x_t)

    plt.plot(time_steps, variance_blur.to("cpu"), label='Blur')
    plt.plot(time_steps, variance_noise.to("cpu"), label='Gaussian Noise')
    plt.legend()
    plt.show()
    plt.plot(time_steps, psnr_blur, label='PSNR Blur')
    plt.plot(time_steps, psnr_noise, label='PSNR Noise')
    plt.legend()
    plt.show()
    break

In [ ]:
from skimage.metrics import structural_similarity as ssim

factor = steps// batch_size
time_steps = range(0, steps, factor)
time_steps = time_steps[:batch_size]
blur_sigmas = [x/10 for x in range(1,9,1)]
t = torch.tensor(time_steps).to(device)
for x, _ in train_loader:
    x = x[0]
    x = x.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)

    all_psnr_blur = []
    all_ssim_blur = []
    for j in blur_sigmas:
        psnr_blur = []
        ssim_blur = []
        x_t = diffusion.blur(x, t,base_sigma=j)
        for i in range(batch_size):
            if i >= len(x_t):
                break
            psnr_blur.append(psnr(x[0], x_t[i]))
            x_0_numpy = x[0].unsqueeze(0).cpu().numpy()[0][0]
            x_t_numpy = x_t[i].unsqueeze(0).cpu().numpy()[0][0]
            data_range = x_0_numpy.max() - x_0_numpy.min()

            ssim_blur.append(ssim(x_0_numpy, x_t_numpy,data_range=data_range))

        all_psnr_blur.append(psnr_blur)
        all_ssim_blur.append(ssim_blur)

    x_t = diffusion.gaussian_noise(x, t)
    psnr_noise = []
    ssim_noise = []
    for i in range(batch_size):
        if i >= len(x_t):
            break
        psnr_noise.append(psnr(x[0], x_t[i]))
        x_0_numpy = x[0].unsqueeze(0).cpu().numpy()[0][0]
        x_t_numpy = x_t[i].unsqueeze(0).cpu().numpy()[0][0]
        data_range = x_0_numpy.max() - x_0_numpy.min()
        ssim_noise.append(ssim(x_0_numpy, x_t_numpy,data_range=data_range))
    
    # Plot PSNR values
    plt.figure(figsize=(10,6))
    for idx, sigma in enumerate(blur_sigmas):
        plt.plot(time_steps, all_psnr_blur[idx], label=f'PSNR Blur σ={sigma:.2f}')
    plt.plot(time_steps, psnr_noise, label='PSNR Gaussian Noise', linestyle='--')
    plt.xlabel('Time Steps')
    plt.ylabel('PSNR')
    plt.title('PSNR Comparison with input image: Blur vs Gaussian Noise')
    plt.legend()
    plt.show()

    # Plot SSIM values  
    plt.figure(figsize=(10,6))
    for idx, sigma in enumerate(blur_sigmas):
        plt.plot(time_steps, all_ssim_blur[idx], label=f'SSIM Blur σ={sigma:.2f}')
    plt.plot(time_steps, ssim_noise, label='SSIM Gaussian Noise', linestyle='--')
    plt.xlabel('Time Steps')
    plt.ylabel('SSIM')
    plt.title('SSIM Comparison with input image: Blur vs Gaussian Noise') 
    plt.legend()
    plt.show()
    
    break

In [ ]:
factor = steps// batch_size
time_steps = range(0, steps, factor)
time_steps = time_steps[:batch_size]
blur_sigmas = [x/20 for x in range(2,8,1)]
t = torch.tensor(time_steps).to(device)
for x, _ in train_loader:
    x = x[0]
    x = x.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)

    all_psnr_blur = []
    all_ssim_blur = []
    for j in blur_sigmas:
        psnr_blur = []
        ssim_blur = []
        x_t = diffusion.blur(x, t, base_sigma=j)
        for i in range(1, batch_size):
            psnr_blur.append(psnr(x_t[i-1], x_t[i]))
            x_prev_numpy = x_t[i-1].unsqueeze(0).cpu().numpy()[0][0]
            x_curr_numpy = x_t[i].unsqueeze(0).cpu().numpy()[0][0]
            data_range = x_prev_numpy.max() - x_prev_numpy.min()
            ssim_blur.append(ssim(x_prev_numpy, x_curr_numpy, data_range=data_range))

        all_psnr_blur.append(psnr_blur)
        all_ssim_blur.append(ssim_blur)

    x_t = diffusion.gaussian_noise(x, t)
    psnr_noise = []
    ssim_noise = []
    for i in range(1, batch_size):
        psnr_noise.append(psnr(x_t[i-1], x_t[i]))
        x_prev_numpy = x_t[i-1].unsqueeze(0).cpu().numpy()[0][0]
        x_curr_numpy = x_t[i].unsqueeze(0).cpu().numpy()[0][0]
        data_range = x_prev_numpy.max() - x_prev_numpy.min()
        ssim_noise.append(ssim(x_prev_numpy, x_curr_numpy, data_range=data_range))
    
    # Plot PSNR values for different blur sigmas and noise
    plt.figure(figsize=(10,6))
    for idx, sigma in enumerate(blur_sigmas):
        plt.plot(time_steps[1:], all_psnr_blur[idx], label=f'PSNR Blur σ={sigma:.2f}')
    plt.plot(time_steps[1:], psnr_noise, label='PSNR Gaussian Noise', linestyle='--')
    plt.xlabel('Time Steps') 
    plt.ylabel('PSNR')
    plt.title('PSNR Comparison between consecutive images: Blur vs Gaussian Noise')
    plt.legend()
    plt.show()

    # Plot SSIM values for different blur sigmas and noise
    plt.figure(figsize=(10,6))
    for idx, sigma in enumerate(blur_sigmas):
        plt.plot(time_steps[1:], all_ssim_blur[idx], label=f'SSIM Blur σ={sigma:.2f}')
    plt.plot(time_steps[1:], ssim_noise, label='SSIM Gaussian Noise', linestyle='--')
    plt.xlabel('Time Steps')
    plt.ylabel('SSIM')
    plt.title('SSIM Comparison between consecutive images: Blur vs Gaussian Noise')
    plt.legend()
    plt.show()
    
    break

In [ ]:
time_steps = [10,20,50,100,250,500,1000]
blur_sigmas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
result = []
t = torch.tensor(time_steps).to(device)

for x, _ in train_loader:
    x = x[0]
    x = x.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)

    for j in blur_sigmas:
        psnr_blur = []
        ssim_blur = []
        x_t = diffusion.blur(x, t, base_sigma=j)
        for i in range(batch_size):
            if i >= len(x_t):
                print(f"Index {i} is out of bounds for x_t with length {len(x_t)}")
                continue
            psnr_blur.append(psnr(x[0], x_t[i]))
            x_0_numpy = x[0].unsqueeze(0).cpu().numpy()[0][0]
            x_t_numpy = x_t[i].unsqueeze(0).cpu().numpy()[0][0]
            data_range = x_0_numpy.max() - x_0_numpy.min()
            ssim_blur.append(ssim(x_0_numpy, x_t_numpy, data_range=data_range))
        
        result.append((j, psnr_blur, ssim_blur))

    break

# Prepare data for heatmap
import seaborn as sns

psnr_heatmap = np.zeros((len(time_steps), len(blur_sigmas)))
ssim_heatmap = np.zeros((len(time_steps), len(blur_sigmas)))

for idx, (j, psnr_blur, ssim_blur) in enumerate(result):
    psnr_heatmap[:, idx] = psnr_blur
    ssim_heatmap[:, idx] = ssim_blur

# Plot heatmap for PSNR
plt.figure(figsize=(12, 8))
sns.heatmap(psnr_heatmap, annot=True, fmt=".2f", xticklabels=blur_sigmas, yticklabels=time_steps, cmap='viridis', cbar_kws={'label': 'PSNR'})
plt.title('PSNR Heatmap')
plt.xlabel('Blur Sigma')
plt.ylabel('Time Steps')
plt.show()

# Plot heatmap for SSIM
plt.figure(figsize=(12, 8))
sns.heatmap(ssim_heatmap, annot=True, fmt=".2f", xticklabels=blur_sigmas, yticklabels=time_steps, cmap='viridis', cbar_kws={'label': 'SSIM'})
plt.title('SSIM Heatmap')
plt.xlabel('Blur Sigma')
plt.ylabel('Time Steps')
plt.show()



In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Initialize results array
psnr_results = np.zeros((len(t), len(blur_sigmas)))  # Switch dimensions

for x, _ in train_loader:
    x = x[0]
    x = x.unsqueeze(0).repeat(batch_size, 1, 1, 1).to(device)

    for step_idx, step in enumerate(t):  # Iterate over steps first
        previous_x_t = None  # Initialize the previous step tensor
        for sigma_idx, j in enumerate(blur_sigmas):  # Iterate over sigmas
            # Apply blur degradation
            current_x_t = diffusion.blur(x, t=torch.tensor([step]).to(device), base_sigma=j)
            
            # Calculate PSNR between the current and previous sigma
            if previous_x_t is not None:
                psnr_value = psnr(previous_x_t, current_x_t)
                psnr_results[step_idx, sigma_idx] = psnr_value
            
            # Update the previous sigma tensor
            previous_x_t = current_x_t

    break

# Plot heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(psnr_results, annot=True, fmt=".2f", xticklabels=blur_sigmas, yticklabels=t.cpu().numpy(), cmap="viridis")
plt.title("Average PSNR Heatmap (Steps vs Blur Sigma)")
plt.xlabel("Blur Sigma")
plt.ylabel("Steps")
plt.show()

# Fit a GMM to the dataset (for the blur degradation)

In [ ]:
FIT_GMM = True

if FIT_GMM:
    all_blurred = []

    for x, _ in train_loader:
        x = x.to(device)
        batch_size_t = x.size(0)
        
        t = torch.full((batch_size_t,), steps).long().to(device)  # Full degradation (T steps)
        x_blurred = diffusion.degradation(x, t)
        
        all_blurred.append(x_blurred.cpu())

    # Stack all blurred images
    all_blurred = torch.cat(all_blurred, dim=0)  # shape: [N, 1, 28, 28]
    all_blurred = all_blurred.squeeze(1)  # [N, 28, 28]

    all_blurred.shape

In [ ]:
if FIT_GMM:

    X = all_blurred.reshape(all_blurred.shape[0], -1)  # [N, 784]

    X.shape

In [ ]:
import joblib
from sklearn.mixture import GaussianMixture

if FIT_GMM:

    

    # Choose number of components (1–5 is usually enough for MNIST)
    gmm = GaussianMixture(n_components=3, covariance_type='full', random_state=0)
    gmm.fit(X.numpy())  # convert to numpy

    # Save the GMM model
    print("Saving GMM model")
    joblib.dump(gmm, 'gmm_model_steps100_sigma1_6k.pkl')

else:
# Load the GMM model
    print("Loading GMM model")
    gmm = joblib.load('gmm_model_steps100_sigma1_6k.pkl')
    print(gmm)

In [ ]:
# Sample new points from the GMM
samples, _ = gmm.sample(batch_size)  # shape [batch_size, 784]

print(samples.shape)

torch.from_numpy(samples).view(batch_size, 1, 28, 28).shape  # Reshape to [batch_size, 1, 28, 28]

# Reshape back into images
samples = torch.from_numpy(samples).float().view(batch_size, 1, 28, 28).to(device)

# Now use samples as initial x_T
x_prev = samples

# Plot the generated images
plot_images(x_prev)

# Training

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import shutil
import os

DELETE_PREVIOUS_RUN = False

run_name = f'coldDiffusion_{degradation_type}_s{steps}_b{batch_size}_{sampling_type}_{learning_rate:.1e}_e{EPOCHS}_{loss_type}_{len(dataset)}_LIMITED'
if scheduler is not None:
    run_name += f'_scheduler_{scheduler.__class__.__name__}'

if DELETE_PREVIOUS_RUN:
    # Delete the previous run folder if it exists
    if os.path.exists('runs/' + run_name):
        shutil.rmtree('runs/' + run_name)
        print(f"Deleted previous run folder: runs/{run_name}")

else:
    # Add a version number to the run name if it already exists
    version = 1
    while os.path.exists('runs/' + run_name):
        run_name = f'coldDiffusion_{degradation_type}_s{steps}_b{batch_size}_{sampling_type}_{learning_rate:.1e}_e{EPOCHS}_{loss_type}_{len(dataset)}_LIMITED'
        if scheduler is not None:
            run_name += f'_scheduler_{scheduler.__class__.__name__}'
        run_name += f'_v{version}'
        version += 1



writer = SummaryWriter('runs/' + run_name)
print(run_name)

In [ ]:
# Cuda clear cache
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Set the noise function to be used for training

# noise_function = diffusion.vignette
# noise_function = diffusion.pixellate

EPOCHS = 50


for epoch in range(EPOCHS):
    print("Epoch ",epoch)
    train_loss = 0
    for x, _ in tqdm(train_loader):
        optimizer.zero_grad()
        x = x.to(device)
        t = diffusion.sample_timesteps(x.shape[0]).to(device)
        x_t = noise_function(x, t)
        pred = model(x_t, t)
        loss = loss_fn(pred, x)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    val_loss = 0
    model.eval()
    with torch.no_grad(): 
        for x, _ in val_loader:

            #print(type(x))
            #print(x.shape)
            x = x.to(device)
            t = diffusion.sample_timesteps(x.shape[0]).to(device)
            x_t = noise_function(x, t)
            pred = model(x_t, t)
            batch_val_loss = loss_fn(pred, x)
            val_loss += batch_val_loss.item()

            del x, x_t, t, pred, batch_val_loss
        scheduler.step(val_loss/len(val_loader))
        # Sample only every n epochs for opt purposes
        if epoch%3 == 0:
            # Sample 1000 images using the GMM
            sampled_images = []
            for _ in range(1000 // batch_size):
                images = diffusion.sample(model, batch_size=batch_size, initial_image='gmm', gmm=gmm, data_loader=val_loader)
                sampled_images.append(images)
            
            # Concatenate all sampled images
            sampled_images = torch.cat(sampled_images, dim=0)
            
            # Save sampled images to a directory for FID calculations
            fid_dir = "./fid_samples"
            if not os.path.exists(fid_dir):
                os.makedirs(fid_dir)
            
            for i, img in enumerate(sampled_images):
                save_image(img, os.path.join(fid_dir, f'sampled_{i}.png'))
        
    # Save loss
    avg_train_loss = train_loss/len(train_loader)
    avg_val_loss = val_loss/len(val_loader)
    writer.add_scalar('Loss/train', avg_train_loss, epoch)
    writer.add_scalar('Loss/val', avg_val_loss, epoch)
    writer.add_scalar('Learning Rate', optimizer.param_groups[0]['lr'], epoch)

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f} | Learning Rate: {scheduler.get_last_lr()[0]:.10f}")
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)

writer.close()


In [ ]:
torch.save(model.state_dict(), './Models/DM/' + run_name + '.pth')
print("Model saved to ./Models/DM/" + run_name + '.pth')

In [ ]:
# LOAD MODEL
import joblib

LOAD_MODEL = True


if LOAD_MODEL:
    model.load_state_dict(torch.load('./Models/DM/coldDiffusion_blur_s40_b32_gmm_1.0e-04_e100_L1_6000_scheduler_ReduceLROnPlateau.pth'))
    gmm = joblib.load('gmm_model_steps40_sigma1.pkl')

In [ ]:
diff2 = coldDiff()
sampled_images = diff2.sample(model, batch_size=5, initial_image='gmm', gmm=gmm, data_loader=val_loader)
plot_images(sampled_images)

### FID

In [ ]:
from pytorch_fid.inception import InceptionV3
from torchvision.utils import save_image
from torchvision.models import inception_v3
import torch.nn.functional as F
import numpy as np
from scipy.linalg import sqrtm


def calculate_fid(mu1, sigma1, mu2, sigma2):
    diff = mu1-mu2
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1+sigma2- 2*covmean)
    return fid

def get_activations(images, model, batch_size=32, dims=2048, device='cuda'):
    model.eval()
    pred_arr = np.empty((len(images), dims))
    with torch.no_grad():
        for i in tqdm(range(0, len(images), batch_size)):
            start = i
            end = i+batch_size
            batch = images[start:end].to(device)
            #print(batch.size())
            #raise
            pred = model(batch)[0]
            pred = F.adaptive_avg_pool2d(pred, output_size=(1,1))
            pred = pred.squeeze(3).squeeze(2).cpu().numpy()
            pred_arr[start:end] = pred
    return pred_arr

#inception_model = inception_v3(pretrained=True, transform_input=False).to(device)
#inception_model.fc = torch.nn.Identity()
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
inception_model = InceptionV3([block_idx]).to(device)

In [ ]:
device

In [ ]:
# Load model
LOAD_MODEL = True
if LOAD_MODEL:
    model = UNet().to(device)
    model.load_state_dict(torch.load('./Models/DM/coldDiffusion_blur_s40_b32_gmm_1.0e-04_e100_L1_6000_scheduler_ReduceLROnPlateau.pth'))
    print("Model loaded from ./Models/DM/coldDiffusion_blur_s40_b32_gmm_1.0e-04_e100_L1_6000_scheduler_ReduceLROnPlateau.pth")

In [ ]:
original_images = []
generated_images = []

# 

model.eval()
FID_LIMIT = 1000 # Limit the number of images to be generated for FID calculation
for i, (x, t) in tqdm(enumerate(train_loader)):
    if i > FID_LIMIT//x.shape[0]:
        break
    original_images.append(x)
    
    sampled = diffusion.sample(model, batch_size=x.shape[0], initial_image='gmm', gmm=gmm, data_loader=val_loader)
    generated_images.append(sampled)

original_images = torch.cat(original_images, dim=0)
generated_images = torch.cat(generated_images, dim=0)

# Save generated images




In [ ]:
import os
from torchvision.utils import save_image

dir = "./cold_diff_samples_ALL_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

for i, img in enumerate(generated_images):
    save_image(img, os.path.join(dir, f'generated_{i}.png'))

In [ ]:
        
del model

In [ ]:
original_images = torch.cat(original_images, dim=0).to(device)
generated_images = torch.cat(generated_images, dim=0).to(device)

In [ ]:
original_images.size(), generated_images.size()

In [ ]:
# Plot the images
plot_images(original_images[:5])
plot_images(generated_images[:5])

In [ ]:
import torchvision.transforms as transforms
import torch

# Define transform
transform = transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Repeat the single channel

original_transformed = []
generated_transformed = []

# Apply transform
for i in range(len(generated_images)):
    original_transformed.append(transform(original_images[i]))
    generated_transformed.append(transform(generated_images[i]))
    
# Stack back into batches
original_transformed = torch.stack(original_transformed)
generated_transformed = torch.stack(generated_transformed)


In [ ]:
original_transformed.size(), generated_transformed.size()

In [ ]:
import numpy as np

original_activations = get_activations(original_transformed, inception_model, device=device)
generated_activations = get_activations(generated_transformed, inception_model, device=device)

# Calculate mean and covariance of the activations
mu1 = np.mean(original_activations, axis=0)
sigma1 = np.cov(original_activations, rowvar=False)
mu2 = np.mean(generated_activations, axis=0)
sigma2 = np.cov(generated_activations, rowvar=False)

# Calculate FID score
fid_score = calculate_fid(mu1, sigma1, mu2, sigma2)
print('FID score:', fid_score)

# FID using torchmetrics

In [ ]:
original_images[0][0]

In [ ]:
# Normalize the images to 0-255 from -1 to 1
original_images = (original_images + 1) / 2 * 255
generated_images = (generated_images + 1) / 2 * 255

# Convert to uint8
original_images = original_images.type(torch.uint8)
generated_images = generated_images.type(torch.uint8)



In [ ]:
# Break generated images into smaller batches for FID
batch_size = 32  # or even 16 if needed
for i in range(0, len(generated_images), batch_size):
    batch = generated_images[i:i+batch_size]

    fid.update(batch, real=False)

for i in range(0, len(original_images), batch_size):
    batch = original_images[i:i+batch_size]
    fid.update(batch, real=True)

In [ ]:
import torch
from torchmetrics.image.fid import FrechetInceptionDistance

with torch.no_grad():
    # Initialize FID
    fid = FrechetInceptionDistance(feature=2048)

    # Update real images
    fid.update(original_images, real=True)

    # Update generated images
    fid.update(generated_images, real=False)

    # Compute FID
    fid_score = fid.compute()
    print("FID:", fid_score.item())

In [ ]:
import importlib
importlib.reload(torchmetrics.image.fid)

### Use GMM to sample images

In [ ]:
from sklearn.mixture import GaussianMixture

test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

diffusion = coldDiff()

fully_degraded_images = []

for x, _ in test_loader:
    x = x.to(device)
    t = diffusion.sample_timesteps(x.shape[0]).to(device)
    x_t = diffusion.blur(x, t)
    flattened_images = x_t.view(x_t.size(0), -1).detach().cpu().numpy()
    fully_degraded_images.extend(flattened_images)

    
fully_degraded_images = np.array(fully_degraded_images)

gmm = GaussianMixture(n_components=3)
gmm.fit(fully_degraded_images)

In [ ]:
n_samples = 100
sampled_images = gmm.sample(n_samples)[0]
initial_images = torch.tensor(sampled_images).float().to(device)
initial_images = initial_images.view(n_samples, 1, 28, 28)
plot_images(initial_images)

In [ ]:
initial_images.shape

In [ ]:
sampled_intensities = gmm.sample(n_samples)

In [ ]:
len(sampled_intensities)

In [ ]:
import joblib

In [ ]:
joblib.dump(gmm, 'Models/DM/gmm_blur_coldDiff.joblib')